<a href="https://colab.research.google.com/github/IamJamesRooke/ztm_ai_agents/blob/master/AI_Sommelier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install openai-agents -q

In [9]:
%cd /content/drive/MyDrive/ZTM AI Agents Course/AI Sommelier

/content/drive/MyDrive/ZTM AI Agents Course/AI Sommelier


In [10]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('ai_agents_openai')

In [11]:
import asyncio
from uuid import uuid4

from openai import OpenAI
from agents import (
    Agent,
    Runner,
    trace,
    ItemHelpers,
    MessageOutputItem,
    HandoffOutputItem,
    FileSearchTool,
    function_tool,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

client = OpenAI()

Uploading the file to the OpenAI API (`https://platform.openai.com/storage`), because it's a pdf, needs read-binary `(rb)` and have to declare its purpose.

In [23]:
# Upload the file once, reuse everywhere
path = "/content/drive/MyDrive/ZTM AI Agents Course/AI Sommelier/Vinhos baba d_urso.pdf"
upload = client.files.create(
    file=open(path, "rb"),
    purpose="assistants",
)

In [24]:
upload

FileObject(id='file-3wiS8B8ZFyC36fkKtsv2Cg', bytes=86584, created_at=1756927337, filename='Vinhos baba d_urso.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [26]:
vs_file = client.vector_stores.files.create(
    vector_store_id = 'vs_68b89400837c8191a426bf1dd5605e5c',
    file_id = upload.id
)

# AI Sommelier Agent

In [29]:
# Define the instructions
RECOMMENDED_PROMPT_PREFIX

'# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n'

In [31]:
instructions = f"""{RECOMMENDED_PROMPT_PREFIX}
You are the restaurant Bitte's AI Sommelier:
Your task is to provide the user with a wine recommendation for their dish.
Use FileSearchTool to pick up to 3 wines from the vector store.
If the interaction has nothing to do with the wine, do the handoff.
"""

vs_id = "vs_68b89400837c8191a426bf1dd5605e5c"

In [35]:
# Define escalate to boss function
@function_tool(
    name_override = "escalate_to_boss",
    description_override="Escalate to the boss if guest talks about wine.")

async def escalate_to_boss(request: str) -> str:
  print(f"Escalating to boss: {request}")
  return "Boss will be here soon."

In [36]:
sommelier = Agent(
    name = "Bitte AI Sommelier",
    model = "gpt-4.1-mini",
    instructions = instructions,
    tools = [
        FileSearchTool(vector_store_ids = [vs_id], max_num_results = 3),
        escalate_to_boss
    ]
)

In [39]:
# Build the chat interactions
async def chat():

  # store conversation
  history = []

  # while loop
  while True:
    user = input("User: ")
    if user.lower() in {"exit", "quit", "q"}:
      break

    history.append({"role": "user", "content": user})

    # Apply tracing to the conversation
    with trace("Bitte Sommelier", group_id = uuid4().hex):
      run = await Runner.run(sommelier, history)

    stop_chat = False

    for itm in run.new_items:
      if isinstance(itm, MessageOutputItem):
        print("Sommelier", ItemHelpers.text_message_output(itm))

      elif isinstance(itm, HandoffOutputItem):
        print("Boss was summoned.")
        stop_chat = True

    if stop_chat:
      break

    history = run.to_input_list()

# Run the chat
await chat()

User: What pairs well with a shit flavored ice cream cone?
Sommelier A "shit flavored" ice cream cone is a very unusual and strong flavor. For something with an intense or offbeat flavor profile, I would generally recommend wines that can either contrast or complement the experience:

1. A sparkling wine or Champagne - The bubbles and acidity will help cleanse the palate after each bite, cutting through strong flavors.
2. A crisp and acidic white wine such as a Sauvignon Blanc or Alvarinho - These wines have citrus and herbal notes that can refresh the palate.
3. A light and fruity rosé - The bright fruit flavors and moderate acidity help balance something heavy or unpleasant.

From the list available, some options that could potentially work include:
- Espumante V. Lafayette Blanc de Blancs (sparkling Chardonnay)
- Branco Minho e Douro Litoral Alvarinho Palácio da Brejoeira
- Rosé França Chateau Roubine Premium (Grenache, Cinsault)

These wines are generally fresh, high in acidity, an